# İçerik Temelli Tavsiye Sistemi
---

## İş Problemi 
---
Yeni kurulmuş bir online film izleme platformu kullanıcılarına film önerilerinde bulunmak istemektedir.

Kullanıcıların login oranı çok düşük olduğu için kullanıcı alışkanlıklarını toplayamamaktadır. Bu sebeple iş birlikçi filtreleme yöntemleri ile ürün önerileri geliştirememektedir.

Fakat kullanıcıların tarayıcıdaki izlerinden hangi filmleri izlediklerini bilmektedir. Bu bilgiye göre film önerilerinde bulununuz.


## Veri Seti Hikayesi
---
movies_metadata.csv veri seti 45000 film ile ilgili temel bilgileri barındırmaktadır.

Uygulama kapsamında film açıklamalarını içeren "overview" değişkeni ile çalışılacaktır.

## TF-IDF Matrisinin Oluşturulması

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/Users/huseyinefkanalp/Desktop/DataScience/DataSets/movies_metadata.csv", low_memory = False)

In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df.shape

(45466, 24)

In [5]:
df["overview"].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [6]:
tfidf = TfidfVectorizer(stop_words="english")

In [7]:
#Overiwiev null olanlar
df[df["overview"].isnull()].head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,0.745542,/k6ODtR38dKEfuzSGjggr8KDyAF4.jpg,"[{'name': 'Iwerks Entertainment', 'id': 70801}]","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0
300,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,161495,tt0114296,sv,Roommates,NaN,3.395867,/hvHNlMvWS2GBt7RR971bJ3k4bJc.jpg,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-03-01,12400000.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Roommates,False,6.4,7.0
634,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,287305,tt0117312,de,Peanuts – Die Bank zahlt alles,NaN,0.066123,/wpk30SvRHmjC2plgKHZXxG0FlKd.jpg,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-21,0.0,NaN,[],Released,NaN,Peanuts – Die Bank zahlt alles,False,4.0,1.0
635,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,339428,tt0116485,de,Happy Weekend,NaN,0.002229,NaN,"[{'name': 'Senator Film Produktion', 'id': 191}]","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Happy Weekend,False,0.0,0.0
641,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,10801,tt0117788,de,Das Superweib,NaN,0.821299,/AbhMKCh3fV5PY2B9uSPF1DWEvq2.jpg,"[{'name': 'Constantin Film', 'id': 47}]","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-06,0.0,86.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The Superwife,False,5.3,7.0


In [8]:
#nullardan kurtulmak için
df["overview"] = df["overview"].fillna("")

In [9]:
tfidf_matrix = tfidf.fit_transform(df["overview"])

In [10]:
tfidf_matrix.shape

(45466, 75827)

In [11]:
#tfidf.get_feature_names() bütün kelimeleri getirir

In [12]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Cosine Similarity Matrisinin Oluşturulması

In [13]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [15]:
cosine_sim.shape

(45466, 45466)

In [17]:
#burada 1.indexdeki filmin diğer bütün filmler ile benzerlik skorları var
cosine_sim[1]

array([0.01504121, 1.        , 0.04681953, ..., 0.        , 0.02198641,
       0.00929411])

## Benzerliklere Göre Önerilerin Yapılması

In [19]:
#her bir filmin ismi ve bu filmlerin index bilgisi var
indices = pd.Series(df.index, index=df["title"])
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [20]:
indices.index.value_counts()

Cinderella              11
Hamlet                   9
Alice in Wonderland      9
Beauty and the Beast     8
Les Misérables           8
                        ..
Cluny Brown              1
Babies                   1
The Green Room           1
Captain Conan            1
Queerama                 1
Name: title, Length: 42277, dtype: int64

In [21]:
# Çoklama problemi
indices = indices[~indices.index.duplicated(keep = "last")]
indices.index.value_counts()

Toy Story                   1
Russell Madness             1
Attack of the Sabretooth    1
The Millennials             1
X/Y                         1
                           ..
Wife! Be Like a Rose!       1
Adelheid                    1
PEEPLI [Live]               1
The Moth                    1
Queerama                    1
Name: title, Length: 42277, dtype: int64

In [23]:
movie_index= indices["Sherlock Holmes"]

In [25]:
similarity_scores = pd.DataFrame(cosine_sim[movie_index],columns=["score"])
similarity_scores.head()

,score
0,0.000000
1,0.003928
2,0.004768
3,0.000000
4,0.000000


In [31]:
movie_indices = similarity_scores.sort_values("score", ascending = False)[1:11].index

In [32]:
df["title"].iloc[movie_indices]

34737    Приключения Шерлока Холмса и доктора Ватсона: ...
14821                                    The Royal Scandal
34750    The Adventures of Sherlock Holmes and Doctor W...
9743                           The Seven-Per-Cent Solution
4434                                        Without a Clue
29706                       How Sherlock Changed the World
18258                   Sherlock Holmes: A Game of Shadows
24665     The Sign of Four: Sherlock Holmes' Greatest Case
6432                   The Private Life of Sherlock Holmes
29154                          Sherlock Holmes in New York
Name: title, dtype: object

## Çalışmanın Scriptini Hazırlama

In [33]:
def content_based_recommender(title, cosine_sim, df):
    #İndexleri oluşturma
    indices = pd.Series(df.index, index=df["title"])
    indices = indices[~indices.index.duplicated(keep="last")]
    #Title'ın indexini yakalama
    movie_index = indices[title]
    #title'a göre benzerlik skoru hesaplama
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    #kendisi hariç ilk 10 film getirme
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return df["title"].iloc[movie_indices]

In [34]:
content_based_recommender("Toy Story", cosine_sim, df)

15348                                     Toy Story 3
2997                                      Toy Story 2
10301                          The 40 Year Old Virgin
24523                                       Small Fry
23843                     Andy Hardy's Blonde Trouble
29202                                      Hot Splash
43427                Andy Kaufman Plays Carnegie Hall
38476    Superstar: The Life and Times of Andy Warhol
42721    Andy Peters: Exclamation Mark Question Point
8327                                        The Champ
Name: title, dtype: object

In [36]:
content_based_recommender("The Matrix", cosine_sim, df)

44161                        A Detective Story
44167                              Kid's Story
44163                             World Record
33854                                Algorithm
167                                    Hackers
20707    Underground: The Julian Assange Story
6515                                  Commando
24202                                 Who Am I
22085                           Berlin Express
9159                                  Takedown
Name: title, dtype: object

In [38]:
content_based_recommender("The Godfather", cosine_sim, df)

1178               The Godfather: Part II
44030    The Godfather Trilogy: 1972-1990
1914              The Godfather: Part III
23126                          Blood Ties
11297                    Household Saints
34717                   Start Liquidation
10821                            Election
38030            A Mother Should Be Loved
17729                   Short Sharp Shock
26293                  Beck 28 - Familjen
Name: title, dtype: object

In [40]:
content_based_recommender("Winter Sleep", cosine_sim, df)

34799                      Pardon
34821           Borderline Normal
32888                Tiger Orange
18570              Beautiful Kate
4020     The Million Dollar Hotel
5807                About Schmidt
33284                    Airborne
2887                Train of Life
33246        Hotel Transylvania 2
35018       Hook, Line and Sinker
Name: title, dtype: object